# AiDM Assignment 2
##  Locality Sensitive Hashing for Netflix Data

###  Load Data
First we read in the data

In [40]:
import numpy as np
data_origin = np.load('../user_movie.npy')

### Shingling
In the interest of memory conservation (only stoing non-zero elements) and efficient rearrangement, we store the data as ones in indicies according to their user ID (column) and rated movies (rows, 1 if rated 0 if not) 

In [ ]:
from scipy.sparse import csc_matrix
users_movies = csc_matrix((np.ones(len(data_origin)), (data_origin[:,1], data_origin[:,0])), shape=(np.max(data_origin[:,1])+1, np.max(data_origin[:,0])+1))

### Signature Matrix
Now we create a random permutations of movie indicies and rearrange our matrix to match. The first non-zero element in the new arrangement will be an element in the user's signature. We will start using 100 permutations, so that our signatures for each user are of length 100.

In [ ]:
import time
tic = time.clock()
signature_length = 10
signature_matrix = np.zeros((signature_length, np.max(data_origin[:,0])+1))
permutation_indicies = np.arange(np.max(data_origin[:,1])+1)
users_movies_copy = np.copy(users_movies)
for i in range(signature_length):
    np.random.shuffle(permutation_indicies)
    users_movies_copy = users_movies[permutation_indicies,:]
    signature_matrix[i,:]= (users_movies_copy!=0).argmax(axis=0)
toc = time.clock()
print('Gee-whiz, it only took', int(toc - tic), 'seconds to make the signature matrix!')

Now let's test to make sure we made our signture matrix correctly. For the last signature in the matrix for the first user, is the index actually the first non-zero entry in the permutation_indicies arrangement?

In [ ]:
for j in permutation_indicies[1+np.arange(int(signature_matrix[signature_length-1,0]))]:
    print('Index:', j, ', Entry:', users_movies[j,0])

Yep, that is indeed the first non-zero entry in this ordering!

###  Partition Into Bands
Next we cut the signature matrix into 5 bands (to start),

In [ ]:
bands = 20